In [2]:
import torch
from typing import Any, Dict, Tuple

OBJ_PATH = "../exemplar_prototype/coco/coco_ins10_.pth"
ATTR_PATH = "../exemplar_prototype/coco/coco_ins50_attribute_kmeans15_scale23.pth"

def humanize_bytes(n: int) -> str:
    units = ["B", "KB", "MB", "GB", "TB"]
    i = 0
    x = float(n)
    while x >= 1024 and i < len(units) - 1:
        x /= 1024.0
        i += 1
    return f"{x:.2f} {units[i]}"

def summarize_file(path: str) -> Dict[str, Any]:
    obj = torch.load(path, map_location="cpu")
    stats = {
        "path": path,
        "num_classes": None,          # 将在遍历时推断
        "bytes_total": 0,             # 文件中全部张量（计入的）总字节
        "bytes_per_class": 0,         # 汇总到“第0维是类别”的张量的单类字节
        "detail": []                  # 每个张量的明细
    }

    def visit(name: str, x: Any):
        if torch.is_tensor(x):
            t = x
            print(x.shape)
            # 只统计第0维为类别维度的张量（至少1维）
            if t.ndim >= 1:
                C = t.shape[0]
                # 初始化/校验类数
                if stats["num_classes"] is None:
                    stats["num_classes"] = C
                # 如果文件里不同张量的第0维类数不一致，不强制报错，保守取首次发现的C
                # 仅当该张量第0维等于已记录C时才计入
                if C == stats["num_classes"]:
                    elem_size = t.element_size()  # 每元素字节
                    elems_total = t.numel()
                    bytes_total_tensor = elems_total * elem_size
                    # 每类元素数 = numel / C（默认能整除；若不能整除则保守整除）
                    elems_per_class = elems_total // C
                    bytes_per_class_tensor = elems_per_class * elem_size

                    stats["bytes_total"] += bytes_total_tensor
                    stats["bytes_per_class"] += bytes_per_class_tensor
                    stats["detail"].append({
                        "name": name,
                        "shape": tuple(t.shape),
                        "dtype": str(t.dtype),
                        "element_size": elem_size,
                        "bytes_total_tensor": bytes_total_tensor,
                        "bytes_per_class_tensor": bytes_per_class_tensor
                    })
        elif isinstance(x, dict):
            for k, v in x.items():
                visit(f"{name}.{k}" if name else str(k), v)
        elif isinstance(x, (list, tuple)):
            for i, v in enumerate(x):
                visit(f"{name}[{i}]", v)
        else:
            # 其他类型忽略
            pass

    visit("", obj)
    return stats

def print_summary(stats: Dict[str, Any]):
    path = stats["path"]
    C = stats["num_classes"]
    bytes_total = stats["bytes_total"]
    bytes_per_class = stats["bytes_per_class"]

    print(f"\nFile: {path}")
    if C is None:
        print("  未发现第0维为类别维度的张量（或文件中不含张量）。")
        return

    print(f"  推断类别数 (dim0): {C}")
    print(f"  单类占用：{bytes_per_class} bytes  ({humanize_bytes(bytes_per_class)})")
    print(f"  全量占用（仅统计按类聚合的张量总和）：{bytes_total} bytes  ({humanize_bytes(bytes_total)})")

    # 如果需要总存储（所有类），通常 = bytes_per_class * C（当且仅当所有按类张量的第0维都为C）
    estimated_total = bytes_per_class * C
    # 这与 bytes_total 一致当且仅当所有被计入的张量都严格是 [C, ...] 且没有额外非按类的张量
    print(f"  估算全量占用（单类*类数）：{estimated_total} bytes  ({humanize_bytes(estimated_total)})")

    # 可选：打印每个张量的明细
    if stats["detail"]:
        print("  明细（每个计入张量）：")
        for d in stats["detail"]:
            print(f"    - {d['name']}: shape={d['shape']}, dtype={d['dtype']}, "
                  f"bytes_per_class={humanize_bytes(d['bytes_per_class_tensor'])}, "
                  f"bytes_total={humanize_bytes(d['bytes_total_tensor'])}")

if __name__ == "__main__":
    obj_stats  = summarize_file(OBJ_PATH)   # 对象级原型文件
    attr_stats = summarize_file(ATTR_PATH)  # 属性级原型文件

    print_summary(obj_stats)
    print_summary(attr_stats)

    # 组合报告（把两个文件的单类占用相加，得到“单类总占用”）
    if obj_stats["num_classes"] and attr_stats["num_classes"]:
        # 为稳妥，取两者一致的类别数（如果不一致，保守取较小者）
        C = min(obj_stats["num_classes"], attr_stats["num_classes"])
        per_class_total = obj_stats["bytes_per_class"] + attr_stats["bytes_per_class"]
        print("\n=== 合并视图（对象 + 属性） ===")
        print(f"  单类总占用：{humanize_bytes(per_class_total)}")
        print(f"  估算全量总占用（按 C={C} 类）：{humanize_bytes(per_class_total * C)}")

torch.Size([65, 2048])
torch.Size([65, 15, 2048])

File: ../exemplar_prototype/coco/coco_ins10_.pth
  推断类别数 (dim0): 65
  单类占用：8192 bytes  (8.00 KB)
  全量占用（仅统计按类聚合的张量总和）：532480 bytes  (520.00 KB)
  估算全量占用（单类*类数）：532480 bytes  (520.00 KB)
  明细（每个计入张量）：
    - : shape=(65, 2048), dtype=torch.float32, bytes_per_class=8.00 KB, bytes_total=520.00 KB

File: ../exemplar_prototype/coco/coco_ins50_attribute_kmeans15_scale23.pth
  推断类别数 (dim0): 65
  单类占用：122880 bytes  (120.00 KB)
  全量占用（仅统计按类聚合的张量总和）：7987200 bytes  (7.62 MB)
  估算全量占用（单类*类数）：7987200 bytes  (7.62 MB)
  明细（每个计入张量）：
    - : shape=(65, 15, 2048), dtype=torch.float32, bytes_per_class=120.00 KB, bytes_total=7.62 MB

=== 合并视图（对象 + 属性） ===
  单类总占用：128.00 KB
  估算全量总占用（按 C=65 类）：8.12 MB
